# Pyber Challenge

### 4.3 Loading and Reading CSV files

In [1]:
# Add Matplotlib inline magic command
%matplotlib inline
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd

# File to Load (Remember to change these)
city_data_to_load = "city_data.csv"
ride_data_to_load = "ride_data.csv"

# Read the City and Ride Data
city_data_df = pd.read_csv(city_data_to_load)
ride_data_df = pd.read_csv(ride_data_to_load)

### Merge the DataFrames

In [2]:
# Combine the data into a single dataset
pyber_data_df = pd.merge(ride_data_df, city_data_df, how="left", on=["city", "city"])

# Display the data table for preview
pyber_data_df.head()

,city,date,fare,ride_id,driver_count,type
0,Lake Jonathanshire,1/14/19 10:14,13.83,5.739410e+12,5,Urban
1,South Michelleport,3/4/19 18:24,30.24,2.343910e+12,72,Urban
2,Port Samanthamouth,2/24/19 4:29,33.44,2.005070e+12,57,Urban
3,Rodneyfort,2/10/19 23:22,23.44,5.149250e+12,34,Urban
4,South Jack,3/6/19 4:28,34.58,3.908450e+12,46,Urban


## Deliverable 1: Get a Summary DataFrame 

In [1]:
#  1. Get the total rides for each city type
total_rides_type = pyber_data_df.groupby("type").count()["ride_id"]
total_rides_type.head()


NameError: name 'pyber_data_df' is not defined

In [4]:
# 2. Get the total drivers for each city type
total_drivers_type = city_data_df.groupby("type").sum()["driver_count"]
total_drivers_type.head()

In [5]:
#  3. Get the total amount of fares for each city type
total_fares_type = pyber_data_df.groupby("type").sum()["fare"]
total_fares_type.head()

In [6]:
#  4. Get the average fare per ride for each city type. 
average_fare_ride = total_fares_type / total_rides_type
average_fare_ride

In [7]:
# 5. Get the average fare per driver for each city type. 
average_fare_driver = total_fares_type / total_drivers_type
average_fare_driver

In [8]:
#  6. Create a PyBer summary DataFrame. 
pyber_summary_df = pd.DataFrame({
    "Total Rides": total_rides_type,
    "Total Drivers": total_drivers_type,
    "Total Fares": total_fares_type,
    "Average Fare per Ride": average_fare_ride,
    "Average Fare per Driver": average_fare_driver})
pyber_summary_df

In [9]:
#  7. Cleaning up the DataFrame. Delete the index name
pyber_summary_df.index.name = None

pyber_summary_df

In [10]:
#  8. Format the columns.
pyber_summary_df["Total Fares"] = pyber_summary_df["Total Fares"].map("${:,.2f}".format)
pyber_summary_df["Average Fare per Ride"] = pyber_summary_df["Average Fare per Ride"].map("${:,.2f}".format)
pyber_summary_df["Average Fare per Driver"] = pyber_summary_df["Average Fare per Driver"].map("${:,.2f}".format)
pyber_summary_df

## Deliverable 2.  Create a multiple line plot that shows the total weekly of the fares for each type of city.

In [11]:
# 1. Read the merged DataFrame
pyber_data_df.head()


In [12]:
# 2. Using groupby() to create a new DataFrame showing the sum of the fares 
#  for each date where the indices are the city type and date.
total_fare_type_date = pyber_data_df.groupby(["type","date"]).sum()["fare"]
total_fare_type_date

In [13]:
# 3. Reset the index on the DataFrame you created in #1. This is needed to use the 'pivot()' function.
# df = df.reset_index()
total_fare_type_date = total_fare_type_date.reset_index()
total_fare_type_date

In [14]:
# 4. Create a pivot table with the 'date' as the index, the columns ='type', and values='fare' 
# to get the total fares for each type of city by the date. 
fares_date_type_pivot = total_fare_type_date.pivot(index="date", columns="type", values="fare")
fares_date_type_pivot

In [15]:
# 5. Create a new DataFrame from the pivot table DataFrame using loc on the given dates, '2019-01-01':'2019-04-29'.
fares_date_type_jan_apr = fares_date_type_pivot.loc[(fares_date_type_pivot.index >= "2019-01-01") & 
                                (fares_date_type_pivot.index <= "2019-04-29")]
fares_date_type_jan_apr


In [16]:
# 6. Set the "date" index to datetime datatype. This is necessary to use the resample() method in Step 8.
# df.index = pd.to_datetime(df.index)
fares_date_type_jan_apr.index = pd.to_datetime(fares_date_type_jan_apr.index)
fares_date_type_jan_apr

In [17]:
# 7. Check that the datatype for the index is datetime using df.info()
fares_date_type_jan_apr.info()
DatetimeIndex: 2196 entries, 2019-01-01 00:08:16 to 2019-04-28 19:35:03
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Rural     114 non-null    float64
 1   Suburban  573 non-null    float64
 2   Urban     1509 non-null   float64
dtypes: float64(3)
memory usage: 68.6 KB

In [18]:
# 8. Create a new DataFrame using the "resample()" function by week 'W' and get the sum of the fares for each week.
fares_week_type_jan_apr = fares_date_type_jan_apr.resample("W").sum()
fares_week_type_jan_apr

In [19]:
# 8. Using the object-oriented interface method, plot the resample DataFrame using the df.plot() function. 

# Import the style from Matplotlib.
from matplotlib import style
# Use the graph style fivethirtyeight.
style.use('fivethirtyeight')

fig, ax = plt.subplots(figsize=(15,6))
fares_week_type_jan_apr.plot(kind="line", ax=ax, color=["gold", "lightskyblue", "lightcoral"])
ax.set_title("Total Fare by City Type")
ax.set_ylabel("Fare ($USD)")
ax.set_xlabel(" ")
lgnd = plt.legend(fontsize="12", mode="Expanded", loc=(0.8,0.5), title="City Types")

# Save Figure
plt.savefig("analysis/PyBer_fare_summary.png")

plt.show()